In [ ]:
import warnings

warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2

import base64
import datetime
import io
import json
import sys
import textwrap
from pathlib import Path
from typing import List, Union

import basedosdados as bd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import google.generativeai as genai
from langchain import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI, chat_models
from pydantic import BaseModel, Field

from IPython.display import Markdown
from PIL import Image, ImageDraw, ImageFont

sys.path.insert(0, "../../")
from utils_sheets import save_data_in_sheets

bd.config.billing_project_id = 'rj-escritorio-dev'
bd.config.from_file = True
bd.__version__


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


from vision_ai_api import APIVisionAI

import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
# GOOGLE_API_KEY = ""
PROJECT_ID = "rj-escritorio-dev"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
with open("secrets.json") as f:
    s = json.load(f)


vision_ai_api = APIVisionAI(
    username=s.get("username"),
    password=s.get("password"),
)

- storage images: https://console.cloud.google.com/storage/browser/datario-public/flooding_detection?project=datario
- imagens figma: https://www.figma.com/file/Qv89NLopXS60Lqf3XfTZiN/Untitled?type=design&node-id=3-44&mode=design&t=3a4g8D4QLiDQ8f3i-0
- langchain ref: https://python.langchain.com/docs/integrations/platforms/google


In [ ]:
def get_image_link_from_storage():
    dataset_id = "flooding_detection"
    table_id = "classified_images"
    st = bd.Storage(dataset_id=dataset_id, table_id=table_id)
    blobs = (
        st.client["storage_staging"]
        .bucket("datario-public")
        .list_blobs(prefix=f"{dataset_id}/{table_id}")
    )

    url_list = []
    for blob in blobs:
        url = str(blob.public_url)
        if "." in url.split("/")[-1]:
            url_list.append(url)
    return url_list


def get_urls_and_labels():
    urls = get_image_link_from_storage()
    labels = [
        {"path": "images_with_label/flood", "label": True, "object": "alagamento"},
        {"path": "images_with_label/no_flood", "label": False, "object": "alagamento"},
    ]
    data = []
    for url in urls:
        for item in labels:
            if item.get("path") in str(url):
                data.append(
                    {
                        "object": item.get("object"),
                        "label": item.get("label"),
                        "image_url": url,
                    }
                )
    return pd.DataFrame(data)


def balance_and_sample(df, N):
    # Get the minimum count of the two labels
    min_count = min(df["label"].value_counts())

    # Balance the DataFrame
    df_balanced = pd.concat(
        [df[df["label"] == True].head(min_count), df[df["label"] == False].head(min_count)]
    )
    df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
    # Sample N rows
    if N > len(df_balanced):
        print(
            f"Requested number of samples ({N}) is more than the available balanced dataset size ({len(df_balanced)})."
        )
        return df_balanced
    return df_balanced.head(N)


def get_image_from_url(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content))
    img.thumbnail((640, 480))
    return img

In [ ]:
from langchain_core.messages import HumanMessage
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, chat_models

from langchain.output_parsers import PydanticOutputParser


class Object(BaseModel):
    object: str = Field(description="The object from the objects")
    label_explanation: str = Field(
        description="Highly detailed visual description of the image given the object context"
    )
    label: Union[bool, str, None] = Field(
        description="Label indicating the condition or characteristic of the object"
    )


class ObjectFactory:
    @classmethod
    def generate_sample(cls) -> Object:
        return Object(
            object="<Object from objects>",
            label_explanation="<Visual description of the image given the object context>",
            label="<Selected label from objects>",
        )


class Output(BaseModel):
    objects: List[Object]


class OutputFactory:
    @classmethod
    def generate_sample(cls) -> Output:
        return Output(objects=[ObjectFactory.generate_sample()])


class OutputFactory:
    @classmethod
    def generate_sample(cls) -> Output:
        return Output(objects=[ObjectFactory.generate_sample()])


def get_parser():

    # Create the output parser using the Pydantic model
    output_parser = PydanticOutputParser(pydantic_object=Output)

    # Valid JSON string
    output_example_str = str(OutputFactory().generate_sample().dict()).replace("'", '"')

    output_example_str = textwrap.dedent(output_example_str)
    output_example = output_parser.parse(output_example_str)
    output_example_parsed = json.dumps(output_example.dict(), indent=4)

    output_schema = json.loads(output_parser.pydantic_object.schema_json())
    output_schema_parsed = json.dumps(output_schema, indent=4)

    return output_parser, output_schema_parsed, output_example_parsed


def gemini_pro_vision_langchain(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro-vision",
        google_api_key=GOOGLE_API_KEY,
        max_output_token=max_output_token,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        stream=True,
    )

    content = [
        {"type": "text", "text": prompt},
        {"type": "image_url", "image_url": image_url},
    ]
    message = HumanMessage(content=content)
    return llm.invoke([message])


def gemini_pro_vision_google(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    image_response = requests.get(image_url)
    image = Image.open(io.BytesIO(image_response.content))
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel("gemini-pro-vision")
    responses = model.generate_content(
        contents=[prompt, image],
        generation_config={
            "max_output_tokens": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
        },
        stream=True,
    )

    responses.resolve()
    ai_response = responses.text
    return ai_response


def gemini_pro_vision_vertex(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    image_response = requests.get(image_url)
    image_type = image_url.split(".")[-1]
    model = GenerativeModel("gemini-pro-vision")
    responses = model.generate_content(
        contents=[prompt, Part.from_data(image_response.content, f"image/{image_type}")],
        generation_config={
            "max_output_tokens": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
        },
    )
    ai_response = responses.text
    return ai_response


def predict_and_save(
    save_data,
    output_parser,
    image_url,
    prompt,
    max_output_token,
    temperature,
    top_k,
    top_p,
    experiment_name,
    experiment_datetime,
    predictions_path,
):
    response = gemini_pro_vision_vertex(
        image_url=image_url,
        prompt=prompt,
        max_output_token=max_output_token,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )

    try:
        response_parsed = output_parser.parse(response)
        response_parsed = response_parsed.dict()
    except Exception as e:
        response_parsed = response

    save_data_in_sheets(
        save_data=save_data,
        data={
            "prompt": prompt,
            "max_output_token": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "experiment_name": experiment_name,
            "experiment_datetime": experiment_datetime,
            "true_object": "",
            "response": response_parsed,
            "image_url": image_url,
            "image": f'=IMAGE("{image_url}")',
        },
        data_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=436224340",
        prompt_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1779223884",
    )

    pd.DataFrame([{"image_url": image_url}]).to_csv(
        path_or_buf=predictions_path,
        index=False,
        header=not predictions_path.exists(),
        mode="a",
    )

    return response, response_parsed

In [ ]:
def explode_df(dataframe, column_to_explode, prefix=None):
    df = dataframe.copy()
    exploded_df = df.explode(column_to_explode)
    new_df = pd.json_normalize(exploded_df[column_to_explode])

    if prefix:
        new_df = new_df.add_prefix(f"{prefix}_")

    df.drop(columns=column_to_explode, inplace=True)
    new_df.index = exploded_df.index
    result_df = df.join(new_df)

    return result_df


def get_objetcs_labels_df(objects, keep_null=False):
    objects_df = objects.rename(columns={"id": "object_id"})
    objects_df = objects_df[["name", "object_id", "labels"]]
    labels = explode_df(objects_df, "labels")
    if not keep_null:
        labels = labels[~labels["value"].isin(["null"])]
    labels = labels.rename(columns={"label_id": "label"})
    labels = labels.reset_index(drop=True)
    return labels


with open("secrets.json") as f:
    s = json.load(f)

vision_api = APIVisionAI(
    username=s.get("username"),
    password=s.get("password"),
)


def get_prompt_template():
    objects = pd.DataFrame(vision_api._get_all_pages(path="/objects"))
    labels = get_objetcs_labels_df(objects, keep_null=True)

    data = vision_api._get_all_pages(path="/prompts")
    prompt_parameters = data[0]
    prompt_text = prompt_parameters.get("prompt_text")
    prompt_objects = prompt_parameters.get("objects")

    selected_labels_cols = ["name", "criteria", "identification_guide", "value"]
    labels = labels[selected_labels_cols]
    labels = labels[labels["name"].isin(prompt_objects)]
    labels = labels.rename(columns={"name": "object", "value": "label"})
    objects_table_md = labels.to_markdown(index=False)
    # prompt_text =  prompt_text.replace("{objects_table_md}", objects_table_md)

    prompt_parameters["prompt_text"] = prompt_text
    prompt_parameters["objects_table_md"] = objects_table_md

    return prompt_parameters


def get_urls_from_path(url_path):
    urls = get_image_link_from_storage()
    data = []
    for url in urls:
        if url_path in str(url):
            data.append(
                {
                    "image_url": url,
                }
            )
    return pd.DataFrame(data)


def get_objects_table_from_sheets(
    url: str = "https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1672006844",
):
    request_url = url.replace("edit#gid=", "export?format=csv&gid=")
    response = requests.get(request_url)
    dataframe = pd.read_csv(io.StringIO(response.content.decode("utf-8")), dtype=str)
    dataframe["label"] = dataframe["label"].fillna("null")
    dataframe = dataframe[dataframe["use"] == "1"]
    dataframe = dataframe.drop(columns=["use"])

    objects_table_md = dataframe.to_markdown(index=False)

    objects_labels = (
        dataframe[["object", "label"]]
        .groupby(by=["object"], sort=False)["label"]
        .apply(lambda x: ", ".join(x))
        .reset_index()
    )
    objects_labels["label"] = objects_labels["label"].str.replace("true, false", "bool")

    objects_labels_md = objects_labels.to_markdown(index=False)
    objects_labels_md = objects_labels_md
    return objects_table_md, objects_labels_md

In [ ]:
def get_prompt(prompt_parameters, prompt_template=None, objects_table_md=None):

    if not prompt_template:
        prompt_template = prompt_parameters.get("prompt_text")

    _, output_schema, output_example = get_parser()

    if not objects_table_md:
        objects_table_md = prompt_parameters.get("objects_table_md")

    filled_prompt = (
        prompt_template.replace("                        ", "")
        .replace("{objects_table_md}", objects_table_md)
        .replace("{output_schema}", output_schema)
        .replace("{output_example}", output_example)
    )

    return filled_prompt, prompt_template


backup = """

"""


prompt_text_local = """

## Role: Urban Road Image Analyst

#### Expertise and Responsibilities:
As an Expert Urban Road Image Analyst, you specialize in interpreting CCTV images to assess various conditions on urban roads. Your expertise includes the detection of image data loss or corruption, as well as analyzing.


#### Key Expertise Areas:
- **Image Data Integrity Analysis:** Expertise in identifying signs of image data loss or corruption, such as uniform grey or green color distortions.
- **Urban Road Condition Assessment:** Proficient in evaluating road conditions and potential hazards unrelated to specific environmental factors.
- **Visual Data Interpretation:** Skilled in analyzing visual data from CCTV images, recognizing patterns and indicators that reflect road conditions and safety issues.

#### Skills:
- **Analytical Prowess:** Exceptional ability to analyze complex visual data, detecting subtle indicators of road-related challenges.
- **Detail-Oriented Observation:** Keen observational skills for identifying minute details in CCTV footage that signify changes in road conditions.


----

### Input

- **Data Provided**: A CCTV image.

### Objects Table 

- **Guidance**: Use the table below for object classification, adhering to the specified criteria and identification guides.

{objects_table_md}

### Scenarios examples:

- Example 1: Dry Road with Clear Traffic
```json
{
    "objects": [
        {
            "object": "image_corrupted",
            "label_explanation": "Image is clear, no distortion or data loss.",
            "label": "false"
        },
        {
            "object": "image_description",
            "label_explanation": "Urban road in daylight with vehicles, clear weather.",
            "label": "null"
        },
        {
            "object": "rain",
            "label_explanation": "Road surface is dry, no signs of water.",
            "label": "false"
        },
        {
            "object": "water_level",
            "label_explanation": "No water present, road surface completely dry.",
            "label": "low"
        },
        {
            "object": "traffic",
            "label_explanation": "Traffic is flowing smoothly, no hindrance observed.",
            "label": "easy"
        },
        {
            "object": "road_blockade",
            "label_explanation": "Road is completely free of obstructions.",
            "label": "free"
        }
    ]
}
```

- Example 2: Partially Flooded Road with Moderate Obstructions
```json
{
    "objects": [
        {
            "object": "image_corrupted",
            "label_explanation": "Slight blurriness in the image, but generally clear.",
            "label": "true"
        },
        {
            "object": "image_description",
            "label_explanation": "Moderate traffic on an urban road with visible puddles.",
            "label": "null"
        },
        {
            "object": "rain",
            "label_explanation": "Puddles observed on parts of the road.",
            "label": "true"
        },
        {
            "object": "water_level",
            "label_explanation": "Water covers some parts of the road, forming puddles.",
            "label": "medium"
        },
        {
            "object": "traffic",
            "label_explanation": "Traffic moving slower due to water on road.",
            "label": "moderate"
        },
        {
            "object": "road_blockade",
            "label_explanation": "Partial obstructions due to water, but traffic can pass.",
            "label": "partially"
        }
    ]
}
```

- Example 3: Fully Flooded and Blocked Road
```json
{
    "objects": [
        {
            "object": "image_corrupted",
            "label_explanation": "High quality, clear image with no issues.",
            "label": "false"
        },
        {
            "object": "image_description",
            "label_explanation": "Road completely submerged in water, no traffic visible.",
            "label": "null"
        },
        {
            "object": "rain",
            "label_explanation": "Road is fully covered in water.",
            "label": "true"
        },
        {
            "object": "water_level",
            "label_explanation": "Water level high, road completely submerged.",
            "label": "high"
        },
        {
            "object": "traffic",
            "label_explanation": "Traffic is impossible due to severe flooding.",
            "label": "impossible"
        },
        {
            "object": "road_blockade",
            "label_explanation": "Road is entirely blocked by flooding, impassable.",
            "label": "totally"
        }
    ]
}
```

### Output

**Output Order**

- **Sequence**: Follow this order in your analysis: 
    1. image_corrupted: true or false
    2. image_description: allways null
    3. rain: true or false
    4. water_level: low, medium or high
    5. traffic: easy, moderate, difficult or impossible
    6. road_blockade: free, partially or totally

- **Importance**: Adhering to this sequence ensures logical and coherent analysis, with each step informing the subsequent ones.



**Example Format** 

- Present findings in a structured JSON format, following the provided example.
        
```json
{output_example}
```

- **Requirement**: Each label_explanation should be a 500-word interpretation of the image, demonstrating a deep understanding of the visible elements.

"""

# # PUT PROMPT
# prompt_parameters = vision_ai_api._get('/prompts').get('items')[0]
# prompt_id = prompt_parameters.get('id')
# request_body = {
#   "name": prompt_parameters.get("name"),
#   "model": prompt_parameters.get("model"),
#   "prompt_text": prompt_text_local,
#   "max_output_token": prompt_parameters.get("max_output_token"),
#   "temperature": 0.1,
#   "top_k": prompt_parameters.get("top_k"),
#   "top_p": prompt_parameters.get("top_p")
# }


# r = vision_ai_api._put(
#     path=f'/prompts/{prompt_id}',
#     json_data=request_body
# )
# print(json.dumps(r.json(),  indent=2))


# Markdown(prompt_text_local)

In [ ]:
# df = get_urls_from_path(url_path="images_predicted_as_flood")
objects_table_md, objects_labels_md = get_objects_table_from_sheets()
df = get_urls_from_path(url_path="/")
prompt_parameters = get_prompt_template()

In [ ]:
experiment_name = "test-object-label-table"
experiment_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

max_output_token = prompt_parameters.get("max_output_token")
temperature = 0.6
top_k = prompt_parameters.get("top_k")
top_p = prompt_parameters.get("top_p")
retry = 5

prompt, prompt_template = get_prompt(
    prompt_parameters, prompt_template=prompt_text_local, objects_table_md=objects_table_md
)

In [ ]:
print(len(prompt))
Markdown(prompt)

# print(prompt)

In [ ]:
predictions_path = Path(f"./data/predictions/{experiment_name}__{experiment_datetime}.csv")

if predictions_path.exists():
    predictions = pd.read_csv(predictions_path)
    predictions_list = predictions["image_url"].tolist()
else:
    predictions_list = []

output_parser, output_schema, output_example = get_parser()

for index, row in df.iterrows():
    image_url = row["image_url"]
    retry_count = 0
    while retry_count <= retry:
        try:
            if image_url not in predictions_list:
                _response = None
                _response, response_parsed = predict_and_save(
                    save_data=True,
                    output_parser=output_parser,
                    image_url=image_url,
                    prompt=prompt,
                    max_output_token=max_output_token,
                    temperature=temperature,
                    top_k=top_k,
                    top_p=top_p,
                    experiment_name=experiment_name,
                    experiment_datetime=experiment_datetime,
                    predictions_path=predictions_path,
                )

                print(f"{index} - {len(df)}")
                # print(json.dumps(response_parsed, indent=4))
                # display(get_image_from_url(image_url))
            else:
                print(f"{index} - {len(df)}: already predicted")

            retry_count = retry + 1
        except Exception as e:
            retry_count += 1
            print(f"{index} - {len(df)}: Error.. Retrying:{retry_count}\n {e}\n\n\n\nAI Response:")
            print(_response)

In [ ]:
# ### Objects Tables to Markdown

# prompt_parameters = vision_ai_api._get('/prompts').get('items')[0]
# df = get_objects_table_from_sheets()

# markdown_str = ""
# for object_name in df['object'].unique().tolist():
#     labels_df = df[df['object']==object_name]
#     i = 0
#     for _, row   in labels_df.iterrows():
#         if i==0:
#             markdown_str += f"\nObject: {object_name}\n"

#         markdown_str+=f"""
#             - Label: {row['labels']}:
#                 - Criteria: {row['criteria']}
#                 - Identification guide: {row['identification_guide']}
#         """.replace("            ","")


#         i+=1

# markdown_str = markdown_str

In [ ]:
model = genai.GenerativeModel("gemini-pro-vision")
model.generate_content()

In [ ]:
genai.configure()

In [ ]:
genai.configure()

In [ ]:
camera_id = "001477"
N = 100
for i, image_url in enumerate(
    N * [f"https://storage.googleapis.com/datario-public/vision-ai/staging/{camera_id}.png"]
):
    retry_count = 0
    while retry_count <= retry:
        try:
            response, response_parsed = predict_and_save(
                save_data=True,
                output_parser=output_parser,
                image_url=image_url,
                prompt=prompt,
                max_output_token=max_output_token,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
            )

            print(f"{i}")
            # print(json.dumps(response_parsed, indent=4))
            # display(get_image_from_url(image_url))
            retry_count = retry + 1
        except Exception as e:
            retry_count += 1
            print(f"{i}: Error.. Retrying:{retry_count}\n {e}\n\n\n\nAI Response:")
            print(response.content)

In [ ]:
len(response_parsed)

In [ ]:
output_parser.parse(response_parsed)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
import seaborn as sns


true_values = df_final["flood"].tolist()
predicted_values = df_final["ai_label"].tolist()

# Calculate metrics
accuracy = accuracy_score(true_values, predicted_values)
precision = precision_score(true_values, predicted_values, pos_label=True)
recall = recall_score(true_values, predicted_values, pos_label=True)
f1 = f1_score(true_values, predicted_values, pos_label=True)
conf_matrix = confusion_matrix(true_values, predicted_values)

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


cm = confusion_matrix(true_values, predicted_values)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["False", "True"],
    yticklabels=["False", "True"],
)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()


# resize_factor = 3
# imgs = 10
# time = 1:00
# Accuracy: 0.7
# Precision: 0.6666666666666666
# Recall: 0.5
# F1 Score: 0.5714285714285715

# resize_factor = 1
# imgs = 10
# time = 1:15
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75


# resize_factor = 5
# imgs = 10
# time = 1:20
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75

In [ ]:
df_final["miss"] = np.where(df_final["flood"] == df_final["ai_label"], False, True)
mask = df_final["miss"] == True
miss = df_final[mask]
miss_imgs = miss["base64"].tolist()
miss_ai_labels = miss["ai_label"].tolist()


for base64_image, ai_label in zip(miss_imgs, miss_ai_labels):
    print(f"AI classyfy as: {ai_label}")
    display_img(base64_image)

You are a highly skilled prompt engineering focus in create prompts for CCTV image recognition. Your task is to optimize and refine a provided example prompt.

you output is a diff between the provided prompt and the new optmized prompt

shall we start?
